In [1]:
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import time

from src.tf_smpl.smpl_with_inverse import SMPL

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### getting mesh vertices from .obj files

In [2]:
def get_vertices(mesh_path, with_texture = False):
    with open(mesh_path, "r") as f:
        content = f.readlines()
    vertices = [[np.float32(v_i) for v_i in v.split('\n')[0].split(' ')[1:]] 
                for v in content if v.startswith('v ')]
    vertices = np.array(vertices)
    if with_texture:
        return vertices
    else:
        return vertices[:,:3]

### aligning two sets of mesh-vertices

In [3]:
def mesh_verts_align(verts_1, verts_2, eps = 1e-8):
    """
    this function aligns verts_2 to verts_1
    verts_1, verts_2 are of shape (?,3)
    """
    # finding bounding boxes
    bbox_1_x_min, bbox_1_x_max = np.min(verts_1[:,0]), np.max(verts_1[:,0])
    bbox_1_y_min, bbox_1_y_max = np.min(verts_1[:,1]), np.max(verts_1[:,1])
    bbox_1_z_min, bbox_1_z_max = np.min(verts_1[:,2]), np.max(verts_1[:,2])
    H1 = bbox_1_z_max - bbox_1_z_min
    W1 = bbox_1_y_max - bbox_1_y_min
    D1 = bbox_1_x_max - bbox_1_x_min
    
    bbox_2_x_min, bbox_2_x_max = np.min(verts_2[:,0]), np.max(verts_2[:,0])
    bbox_2_y_min, bbox_2_y_max = np.min(verts_2[:,1]), np.max(verts_2[:,1])
    bbox_2_z_min, bbox_2_z_max = np.min(verts_2[:,2]), np.max(verts_2[:,2])
    H2 = bbox_2_z_max - bbox_2_z_min
    W2 = bbox_2_y_max - bbox_2_y_min
    D2 = bbox_2_x_max - bbox_2_x_min
    
    # get_centers
    center_1 = 0.5* np.array([(bbox_1_x_min + bbox_1_x_max), 
                              (bbox_1_y_min + bbox_1_y_max),
                              (bbox_1_z_min + bbox_1_z_max)])

    center_2 = 0.5* np.array([(bbox_2_x_min + bbox_2_x_max), 
                              (bbox_2_y_min + bbox_2_y_max),
                              (bbox_2_z_min + bbox_2_z_max)])
    
    verts_2 = verts_2 - center_2
    verts_2[:,0] = verts_2[:,0]*(D1/D2+eps) 
    verts_2[:,1] = verts_2[:,1]*(W1/W2+eps)
    verts_2[:,2] = verts_2[:,2]*(H1/H2+eps)
    
    verts_2 = verts_2 + center_1
    
    return verts_2

### saving new vertices to .obj file with faces taken from the reference path

In [4]:
def save_mesh_to_obj(new_verts, reference_path, save_path, textures=False):
    with open(reference_path, "r") as f:
        content = f.readlines()
        lines = content#.split('\n')
        new_content = []
        for i in tqdm(range(len(lines))):
            line = lines[i]
            if not line.startswith('v '):
                new_line = line
            else: 
                verts_part = ' '.join(line.split(' ')[:4])
                if textures:
                    new_line = line.replace(verts_part, 'v '+' '.join([str(val) for val in new_verts[i]]))
                else:
                    new_line = line.replace(verts_part, 'v '+' '.join([str(val) for val in new_verts[i]])+'\n')
            new_content.append(new_line)
    new_content = ''.join(new_content)
    with open(save_path, 'w') as file:
        file.write(new_content)

### make an SMPL model

In [5]:
smpl = SMPL('/Users/test/Desktop/hmr/models/neutral_smpl_with_cocoplus_reg.pkl')



Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



In [6]:
# pifu mesh vertices
pifu_vertices = get_vertices('../../Downloads/result_levon.obj')

# beta_and_theta_1
theta_and_beta_1 = np.load('theta_and_beta_basketball.npy')
theta_1, beta_1 = theta_and_beta_1[:,:72], theta_and_beta_1[:,72:]

In [7]:
# load smpl template from pickle
with open('/Users/test/Desktop/hmr/models/neutral_smpl_with_cocoplus_reg.pkl', 'rb') as f:
    dd = pickle.load(f, encoding="latin-1")

In [8]:
# blend_weights_for_smpl
blend_weights_smpl = dd['weights']

In [9]:
# Note, these smpl vertices corresponds to the same image as pifu vertices
smpl_vertices = smpl.predict_stright(beta_1, theta_1, blend_weights=blend_weights_smpl)

### aligning pifu vertices to the corresponding smpl vertices

In [10]:
# CAUTION: in this special case we need to rotate the pifu mesh manually, but it may be useless in other cases.
pifu_vertices_rotated =  pifu_vertices.copy()
pifu_vertices_rotated[:,1] = -pifu_vertices_rotated[:,1]
pifu_vertices_rotated[:,2] = -pifu_vertices_rotated[:,2]

In [11]:
# aligning to the corresponding smpl vertices
pifu_vertices_aligned = mesh_verts_align(smpl_vertices[0], pifu_vertices_rotated)

### finding blend weights for pifu mesh

In [12]:
def pairwise_dist(A, B):
    # squared norms of each row in A and B
    na = np.sum(np.square(A), axis = 1)
    nb = np.sum(np.square(B), axis =  1)

    # na as a row and nb as a co"lumn vectors
    na = np.reshape(na, [-1, 1])
    nb = np.reshape(nb, [1, -1])

    # return pairwise euclidead difference matrix
    D = np.maximum(na - 2*np.matmul(A, B.T) + nb, 0.0)
    
    return D

In [13]:
# for each vertex v in pifu_vertices find the closest vertex u in smpl_vertices and 
# put blend weights of v as the blend weights of u
start = time()
dists = pairwise_dist(pifu_vertices_aligned, smpl_vertices[0])
print(time()-start)

argmins = np.argmin(dists, axis=-1)

3.022251605987549


In [14]:
blend_weights_pifu = blend_weights_smpl[argmins]

### finding the zero-pose (textured) mesh for pifu

In [15]:
pifu_vertices_zero_pose = smpl.predict_inverse(beta_1, theta_1, blend_weights=blend_weights_pifu, 
                                               binding_verts=pifu_vertices_aligned[None,:,:])

In [16]:
# save pifu zero pose to an .obj file
save_mesh_to_obj(pifu_vertices_zero_pose[0], '../../Downloads/result_levon.obj',
                save_path='pifu_zero_pose.obj', textures=True)

100%|██████████| 127500/127500 [00:00<00:00, 571899.78it/s]


### move pifu mesh to the pose obtained from the parameters beta2, theta2

In [17]:
def get_pifu_mesh(pifu_zero_pose, ref_path='theta_and_beta_thenis.npy', blend_weights=None):
    theta_and_beta = np.load(ref_path)
    theta, beta = theta_and_beta[:,:72], theta_and_beta[:,72:]
    pifu_result_vertices = smpl.predict_stright(beta, theta, blend_weights=blend_weights,
                                                binding_verts=pifu_zero_pose)
    return pifu_result_vertices

In [18]:
pifu_result_vertices = get_pifu_mesh(pifu_vertices_zero_pose, blend_weights=blend_weights_pifu)

In [20]:
# save pifu result pose to an .obj file
save_mesh_to_obj(pifu_result_vertices[0], '../../Downloads/result_levon.obj',
                save_path='pifu_thenis_pose.obj', textures=True)

100%|██████████| 127500/127500 [00:00<00:00, 579144.95it/s]
